<a href="https://colab.research.google.com/github/nb2838/EWB-visualization/blob/Chen/app_0812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run this cell if notebook is executed in Google Colab
# ignore if run in jupyter notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install selenium

In [107]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/bin/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (136 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [108]:
import os
import folium
from folium import FeatureGroup, LayerControl, Map, Marker


import time
import selenium
from selenium import webdriver
import datetime


import re
import pandas as pd 

CSV_EXTENSION  = 'csv'
EXCEL_EXTENSION = 'xls'

datasheets = ["all-ewb","ghana.csv", "morocco.csv", "uganda.csv"]


# ewb object design
# class EWBPoint():
#     def __init__(self, title, latitude, longtitude, descrition):
#         self.name = title
#         self.latitude = latitude
#         self.longitude = longitude
#         self.description = description
    
#     def makePopup(self):
#         pass
##### 

class EWBLayer():
    def __init__(self, name, color, df):
        self.name = name
        self.color = color
        self.df = df
        self.points = []
        self.featureGroup = FeatureGroup(name=name)

class EWBMap():
    """
    Object to reprent map
    Attributes: 
    - TBD
    """

    def __init__(self):
        self.layers = []
        self.map = Map(
            location=[45.372, -121.6972],
            zoom_start=12,
            tiles='Stamen Terrain'
        )
        self.layerNames = []
    
    def makeMapfromLayers(self):
        for layer in self.layers:
            layer.featureGroup.add_to(self.map)
        folium.LayerControl().add_to(self.map)
    
    def recenter(self, string = "all"):
        # center around the town - hardcode location
        if string == 'town':
            x = 45.5236
            y = -122.6750
            self.map.location = [x, y]

        elif string in self.layerNames:
            xMin, xMax, yMin, yMax = [100, -100, 181, -181]
            idx = self.layerNames.index(string)
            layer = self.layers[idx]
            xMin = min(xMin, min(layer.df['latitude']))
            xMax = max(xMax, max(layer.df['latitude']))
            yMin = min(yMin, min(layer.df['longitude']))
            yMax = max(yMax, max(layer.df['longitude']))
            # adjust the map bound according the upper and lower bound of the dataset
            self.map.fit_bounds([[xMin, yMin], [xMax, yMax]])
        
        elif string == 'all':
            xMin, xMax, yMin, yMax = [100, -100, 181, -181]
            for layer in self.layers:
                xMin = min(xMin, min(layer.df['latitude']))
                xMax = max(xMax, max(layer.df['latitude']))
                yMin = min(yMin, min(layer.df['longitude']))
                yMax = max(yMax, max(layer.df['longitude']))
            # adjust the map bound according the upper and lower bound of the dataset
            self.map.fit_bounds([[xMin, yMin], [xMax, yMax]])
        # if the input string does not match any of the options
        else:
            print(f"ERROR: wrong string input: {string}\n, valid inputs are: {self.layerNames} or 'all' or 'town")

#####

        
def determine_extension(filename: str) -> str:
    """
    Returns type of extension constant  associated with a Filename.
    If Extension is not supperted IOError is raised.
    """
    name_pattern = r'[\w\s_]+'
    excel_reg = re.compile(name_pattern + r'.(xls|xlsx|xlsm|xlsb|xls)$')
    csv_reg = re.compile(name_pattern + r'.csv$')
    extension = None
        
    if excel_reg.match(filename):
        extension = EXCEL_EXTENSION
    elif csv_reg.match(filename):
        extension = CSV_EXTENSION
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    
    return extension

    
def get_dataframe(filename: str) -> pd.DataFrame:
    """
    Returns a dataframe from an excel or csv file
    """
    
    extension = determine_extension(filename)
    df = None
    if extension == EXCEL_EXTENSION:
        df = pd.read_excel(filename)
    elif extension == CSV_EXTENSION:
        df = pd.read_csv(filename)
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    
    return df


def get_map(files: list) -> EWBMap:
    """"
    Returns a map object read from a csv or excel file
    """
    map_object = EWBMap()
    for f in files:
        layer = get_layer(f)
        map_object.layers.append(layer)
        map_object.layerNames.append(layer.name)
    map_object.makeMapfromLayers()
    map_object.recenter()
    return map_object


#####
def get_layer(filename: str) -> EWBLayer:
    map_df = get_dataframe(filename)
    layername, color, _= re.split(',|_|\\.',filename.replace(' ', ''))
    layer_object = EWBLayer(layername, color, map_df)

    for row in map_df.itertuples():
        make_popups(layer_object.featureGroup, row.latitude, row.longitude, row.title, 
                    row.date, row.description, row.icon, layer_object.color)

    return layer_object
#####   
    
def make_popups(layer, lat, lon, title, date="", description="", icon="home", color="lightgray"):
    if date == "":
        date = datetime.datetime.now()
    folium.Marker(
        location = [lat, lon],
        icon = folium.Icon(icon=icon, color=color, prefix='fa'),
        tooltip = title,
        popup = folium.Popup(
                    folium.Html('<b>%s</b> <br> <i>%s</i> <br> %s' %(title, date, description), script=True),
                    # min_width=100,
                    max_width=450)
        ).add_to(layer)
    
    
def map_to_html (m, map_name, file_path =""):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    path = file_path+map_name+".html"
    m.save(path)
    return path

# still not working :/
def map_to_png(m, map_name, file_path ="", browser= 'Chrome'):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    fn = map_name+".html"
    m.save(fn)
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    delay =5
    if browser == 'Safari':
        browser = webdriver.Safari()
    elif browser == 'Firefox':
        browser = webdriver.Firefox()
    else:
        browser = webdriver.Chrome()
    options = webdriver.ChromeOptions()
    options.binary_location = "./chromedriver.exe"    #chrome binary location specified here
    options.add_argument("--start-maximized") #open Browser in maximized mode
    options.add_argument("--no-sandbox") #bypass OS security model
    options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    brower = webdriver.Chrome(options=options, executable_path=r'./chromedriver.exe')
    # browser.get('http://google.com/')
    
    browser.get(tmpurl)
    # time.sleep(delay)
    # browser.save_screenshot(file_path+map_name+'.png')
    # browser.quit()



In [ ]:
map_to_html(m, "EWB Map", "")

In [102]:
# change the directory to where you store the notebook and the excel/csv file
%cd /content/drive/My Drive/Colab Notebooks/folium
!ls

/content/drive/My Drive/Colab Notebooks/folium
app.ipynb  chromedriver.exe  sample_blue.xlsx  sampleHTML.html	samplePNG.html


In [112]:
# test cases
sampleMap = get_map(['sample_blue.xlsx', 'vertical_red.xlsx'])
sampleMap.map
# map_to_html(sampleMap.map, 'sampleHTML', '.')
# map_to_png(sampleMap.map, 'samplePNG', '.')